## Part 1

In [1]:
# given a coordinate, a symbol and the map_dict
# returns both positions that are connected to the pipe
def follow_pipe_both_sides(pipe_coord: list, pipe_type: str, map_dict: dict) -> [[],[]]:
    first_dir = map_dict[pipe_type][0]
    secon_dir = map_dict[pipe_type][1]
    return [[pipe_coord[idx] + first_dir[idx] for idx in [0,1]], [pipe_coord[idx] + secon_dir[idx] for idx in [0,1]]]

In [ ]:
# [y,x] = [0,0] is top left
map_dict = {
    '|' : [[1, 0],[-1, 0]],
    '-' : [[0, 1],[0, -1]],
    'L' : [[-1, 0],[0, 1]], 
    'J' : [[-1, 0],[0, -1]], 
    '7' : [[1, 0],[ 0, -1]],
    'F' : [[1, 0],[0, 1]],
    '.' : [[0,0],[0,0]], 
    'S' : [[0,0],[0,0]]

}

pipe_map = []

# generate map, find coordinate of S
with open('Day10_input.txt') as f:
    for idx, line in enumerate(f):
        pipe_map.append(line.strip('\n'))
        if 'S' in line:
            S_pos = [idx, line.find('S')]
            

# find a starting direction from S
neighbours_y = [-1, -1, -1, 0, 1, 1, 1, 0]
neighbours_x = [-1, 0, 1, 1, 1, 0, -1, -1]

starting_pipe = []
for ydx, xdx in zip(neighbours_y, neighbours_x):
    poss_neighb = follow_pipe_both_sides([S_pos[0] +ydx, S_pos[1] + xdx], pipe_map[S_pos[0] +ydx][ S_pos[1] + xdx] , map_dict)
    if S_pos in poss_neighb:
        current_pos = [S_pos[0] +ydx, S_pos[1] + xdx] #first pipe connected to S
        new_pipe_type = pipe_map[current_pos[0]][current_pos[1]] #pipe type of first pipe
        break

diameter = 1
previous_pos = S_pos

while new_pipe_type != 'S':
    #follow both direction from current_pos
    new_pos = follow_pipe_both_sides(current_pos, new_pipe_type, map_dict)
    # remove previous_pos
    new_pos.remove(previous_pos)
    new_pos = new_pos[0]
    # update new_pipe_type, previous_pos, current_pos
    new_pipe_type = pipe_map[new_pos[0]][new_pos[1]]
    previous_pos = [] +current_pos
    current_pos = [] +new_pos
    diameter +=1

print(diameter/2)

## Part 2

In [2]:
# given a list, a char and a position
# returns the list with the updated char in that position
def change_char_in_str(string: str, new_char: str, position: int) -> str:
    string_list = list(string)
    string_list[position]= new_char
    string = "".join(string_list)
    
    return string

In [84]:
# given two map lines, the first with ' ' in place of the pipes that belong to the cycle, and the second being the original
# one, returns the number of chars that are inside the cycle and not part of it
def count_inside_points(modif_line: str, orig_line: str) -> int:
    counter = 0
    is_inside = False
    
    # these are to deal with the cases FJ, l7 -> have a crossing, and F7, LJ -> do not have a crossing
    convex = False
    concave = False
    is_diagonal = False
    for modif_pipe, orig_pipe in zip(modif_line, orig_line):
        if modif_pipe == ' ': #if we are walking on the cycle
            if orig_pipe == '|': # this surely makes us cross it
                is_inside = not is_inside
            
            # these two variables count the parity of convex/concave bumps on the line
            # in my map, S is an 'L'. It can be computed automatically and passed to this function    
            elif orig_pipe in 'SLJ':
                concave = not concave

            elif orig_pipe in 'F7':
                convex = not convex
            
            # convex + concave -> straight diagonal line, counts as crossing
            if concave and convex:
                is_inside = not is_inside
                concave = False
                convex = False
        # if it does not belong to the cycle and we are inside, add 1        
        elif modif_pipe in '.FLJ7|-' and is_inside:
            
            counter += 1
    return counter


In [85]:
# [y,x] = [0,0] is top left
map_dict = {
    '|' : [[1, 0],[-1, 0]],
    '-' : [[0, 1],[0, -1]],
    'L' : [[-1, 0],[0, 1]], 
    'J' : [[-1, 0],[0, -1]], 
    '7' : [[1, 0],[ 0, -1]],
    'F' : [[1, 0],[0, 1]],
    '.' : [[0,0],[0,0]], 
    'S' : [[0,0],[0,0]]

}

pipe_map = []

# generate map, find coordinate of S
with open('Day10_input.txt') as f:
    for idx, line in enumerate(f):
        pipe_map.append(line.strip('\n'))
        if 'S' in line:
            S_pos = [idx, line.find('S')]

# creating copy for '.' counting later
original_map = []+ pipe_map

# find a starting direction from S
neighbours_y = [-1, -1, -1, 0, 1, 1, 1, 0]
neighbours_x = [-1, 0, 1, 1, 1, 0, -1, -1]
starting_pipe = []
for ydx, xdx in zip(neighbours_y, neighbours_x):
    poss_neighb = follow_pipe_both_sides([S_pos[0] +ydx, S_pos[1] + xdx], pipe_map[S_pos[0] +ydx][ S_pos[1] + xdx] , map_dict)
    if S_pos in poss_neighb:
        current_pos = [S_pos[0] +ydx, S_pos[1] + xdx] #first pipe connected to S
        new_pipe_type = original_map[current_pos[0]][current_pos[1]] #pipe type of first pipe
        pipe_map[current_pos[0]] = change_char_in_str(pipe_map[current_pos[0]],  ' ', current_pos[1])
        break


previous_pos = S_pos
# removing 'S' from pipe_map
pipe_map[S_pos[0]] = change_char_in_str(pipe_map[S_pos[0]],  ' ', S_pos[1])

# follows the cycle until we reach back 'S'
while new_pipe_type != 'S':
    #follow both direction from current_pos
    new_pos = follow_pipe_both_sides(current_pos, new_pipe_type, map_dict)
    # remove previous_pos
    new_pos.remove(previous_pos)
    new_pos = new_pos[0]
    # update new_pipe_type, previous_pos, current_pos
    # now the new_pipe_type is computed on original_map, so no issues with S being cancelled in pipe_map
    new_pipe_type = original_map[new_pos[0]][new_pos[1]]
    previous_pos = [] +current_pos
    current_pos = [] +new_pos
    pipe_map[previous_pos[0]] = change_char_in_str(pipe_map[previous_pos[0]],  ' ', previous_pos[1])



point_counter = 0
# for each line, computes how many internal .-|FL7J chars are
for modif_line, orig_line in zip(pipe_map, original_map) :
    point_counter += count_inside_points(modif_line, orig_line)
        
print(point_counter)

483
